In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

RANDOM_SEED = 42

In [ ]:
data_directory = "../../../data/"
train_dataset = f'{data_directory}gesture_train.csv'
test_dataset = f'{data_directory}gesture_test.csv'
model_save_path = './gesture_classifier.pkl'

In [ ]:
NUM_CLASSES = 2
SEQUENCE_FRAMES = 4
MULTI_HAND_LANDMARKS = 128

In [ ]:
X_train = np.loadtxt(train_dataset, delimiter=',', dtype='float32', usecols=list(range(1, (MULTI_HAND_LANDMARKS * SEQUENCE_FRAMES) + 1)))
X_test = np.loadtxt(test_dataset, delimiter=',', dtype='float32', usecols=list(range(1, (MULTI_HAND_LANDMARKS * SEQUENCE_FRAMES) + 1)))

In [ ]:
print(X_train[0])

In [ ]:
y_train = np.loadtxt(train_dataset, delimiter=',', dtype='int32', usecols=(0))
y_test = np.loadtxt(test_dataset, delimiter=',', dtype='int32', usecols=(0))

In [ ]:
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
X_train = np.array(X_train)
print(X_train.shape)

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0]
}

xgb_model = XGBClassifier(tree_method="hist", device="cpu")
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
model = XGBClassifier(**best_params)

In [ ]:
model.fit(
    X_train,
    y_train
)

In [ ]:
# Model evaluation
# val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

model.score(X_test, y_test)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
# y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, Y_pred)

In [ ]:
import pickle

pickle.dump(model, open(model_save_path, 'wb'))

In [ ]:
# Save as a model dedicated to inference
model.save_model(model_save_path)

In [ ]:
loaded_model = XGBClassifier()
loaded_model.load_model(model_save_path)

In [ ]:
loaded_model.score(X_test, y_test)